# Ensuring Consistency

**Activity Overview**: Ensure consistency by identifying and resolving conflicting values across datasets.

## Title: Customer Address Discrepancies

**Task**: Address customer address mismatches between CRM and marketing databases.

**Steps**:
1. Compare customer addresses in the CRM with those in the marketing database.
2. Identify records with conflicting address information.
3. Propose a method to consolidate records with verified addresses.

In [1]:
# Write your code from here
import pandas as pd

def resolve_customer_address_discrepancies(crm_file, marketing_file, customer_id_col='customer_id', address_col_crm='address_crm', address_col_marketing='address_marketing', verified_address_source=None):
    """
    Addresses customer address mismatches between CRM and marketing databases.

    Args:
        crm_file (str): Path to the CRM database CSV file.
        marketing_file (str): Path to the marketing database CSV file.
        customer_id_col (str, optional): Name of the common customer ID column.
                                       Defaults to 'customer_id'.
        address_col_crm (str, optional): Name of the address column in the CRM file.
                                        Defaults to 'address_crm'.
        address_col_marketing (str, optional): Name of the address column in the
                                            marketing file. Defaults to 'address_marketing'.
        verified_address_source (str, optional): Source considered to have the
                                                 verified address ('CRM' or 'Marketing').
                                                 If None, no automatic consolidation is proposed.
                                                 Defaults to None.

    Returns:
        pandas.DataFrame: A DataFrame showing customers with conflicting address
                          information.
        pandas.DataFrame (optional): A DataFrame showing consolidated records
                                     if a verified_address_source is provided.
    """
    try:
        crm_df = pd.read_csv(crm_file)
        marketing_df = pd.read_csv(marketing_file)
    except FileNotFoundError as e:
        print(f"Error: One or both files not found: {e}")
        return None, None

    if customer_id_col not in crm_df.columns or customer_id_col not in marketing_df.columns:
        print(f"Error: Customer ID column '{customer_id_col}' not found in one or both files.")
        return None, None

    if address_col_crm not in crm_df.columns or address_col_marketing not in marketing_df.columns:
        print(f"Error: Address columns '{address_col_crm}' or '{address_col_marketing}' not found.")
        return None, None

    # Merge the two DataFrames on customer_id
    merged_df = pd.merge(crm_df, marketing_df, on=customer_id_col, how='inner', suffixes=('_crm', '_marketing'))

    # Identify records with conflicting address information
    conflicts_df = merged_df[merged_df[address_col_crm] != merged_df[address_col_marketing]][[customer_id_col, address_col_crm, address_col_marketing]]

    print("Records with Conflicting Address Information:")
    print(conflicts_df)

    consolidated_df = None
    if verified_address_source == 'CRM':
        print("\nProposed Consolidation (using CRM address as verified):")
        consolidated_df = conflicts_df[[customer_id_col, address_col_crm]].rename(columns={address_col_crm: 'verified_address'})
    elif verified_address_source == 'Marketing':
        print("\nProposed Consolidation (using Marketing address as verified):")
        consolidated_df = conflicts_df[[customer_id_col, address_col_marketing]].rename(columns={address_col_marketing: 'verified_address'})
    elif verified_address_source:
        print(f"\nNote: '{verified_address_source}' is not a recognized verified address source ('CRM' or 'Marketing'). No automatic consolidation proposed.")
    else:
        print("\nNo verified address source specified. Manual review and consolidation are recommended.")

    return conflicts_df, consolidated_df

# Example Usage:
crm_file = 'crm_data.csv'
marketing_file = 'marketing_data.csv'

# Create sample CSV files
crm_data = {'customer_id': [1, 2, 3, 4],
            'name': ['Alice', 'Bob', 'Charlie', 'David'],
            'address_crm': ['123 Main St', '456 Oak Ave', '789 Pine Ln', '101 Elm Rd']}
crm_df = pd.DataFrame(crm_data)
crm_df.to_csv(crm_file, index=False)

marketing_data = {'customer_id': [1, 2, 3, 5],
                  'email': ['alice@example.com', 'bob@example.com', 'charlie@example.org', 'eve@example.com'],
                  'address_marketing': ['123 Main St', '456 Oak Av', '789 Pine Lane', '555 Willow Dr']}
marketing_df = pd.DataFrame(marketing_data)
marketing_df.to_csv(marketing_file, index=False)

conflicts, consolidated = resolve_customer_address_discrepancies(crm_file, marketing_file)

print("\nConflicting Addresses DataFrame:")
print(conflicts)

if consolidated is not None:
    print("\nProposed Consolidated Addresses DataFrame:")
    print(consolidated)

# Example with a specified verified source (CRM)
conflicts_crm_verified, consolidated_crm_verified = resolve_customer_address_discrepancies(
    crm_file, marketing_file, verified_address_source='CRM'
)
print("\nConflicting Addresses (CRM Verified):")
print(conflicts_crm_verified)
print("\nConsolidated Addresses (CRM Verified):")
print(consolidated_crm_verified)

# Example with a specified verified source (Marketing)
conflicts_marketing_verified, consolidated_marketing_verified = resolve_customer_address_discrepancies(
    crm_file, marketing_file, verified_address_source='Marketing'
)
print("\nConflicting Addresses (Marketing Verified):")
print(conflicts_marketing_verified)
print("\nConsolidated Addresses (Marketing Verified):")
print(consolidated_marketing_verified)

Records with Conflicting Address Information:
   customer_id  address_crm address_marketing
1            2  456 Oak Ave        456 Oak Av
2            3  789 Pine Ln     789 Pine Lane

No verified address source specified. Manual review and consolidation are recommended.

Conflicting Addresses DataFrame:
   customer_id  address_crm address_marketing
1            2  456 Oak Ave        456 Oak Av
2            3  789 Pine Ln     789 Pine Lane
Records with Conflicting Address Information:
   customer_id  address_crm address_marketing
1            2  456 Oak Ave        456 Oak Av
2            3  789 Pine Ln     789 Pine Lane

Proposed Consolidation (using CRM address as verified):

Conflicting Addresses (CRM Verified):
   customer_id  address_crm address_marketing
1            2  456 Oak Ave        456 Oak Av
2            3  789 Pine Ln     789 Pine Lane

Consolidated Addresses (CRM Verified):
   customer_id verified_address
1            2      456 Oak Ave
2            3      789 Pine Ln
Re